Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Introduction to Azure Machine Learning service: Run experiment

In this example, you'll learn how to use Azure Machine Learning for experimentation. The concepts you'll learn about are workspace, experiment and run.

**Run** is a an execution of Python code that does a machine learning task, such as training a model. Within a run you can log metrics and upload results to Azure cloud, to keep track of your experimentation. 
 
In this example, the run is a simple notebook cell, but in subsequent tutorials you can learn how to submit different kinds of runs - hyperparameter tuning, automated machine learning, distributed training - to scalable cloud compute.
 
**Experiment** is a collection of related runs. For example, if you train different models to solve the same problem, you can group the training runs under the same experiment, and later compare their results.
 
**Workspace** is an Azure resource that contains your experiments, models, deployments and cloud compute resources.

To illustrate these concepts, we use a simple example of Monte Carlo simulation to estimate pi. You first connect to your workspace, create an experiment that will contains the different simulation runs, and then launch a run and log the progress on Monte Carlo simulation.

First, let's import the Python packages and load your workspace. When you run *ws = Workspace.from_config* below, you will be prompted to log in to your Azure subscription. Once you are connected to your workspace in Azure cloud, you can start experimenting.

In [1]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle

Run the next cell and follow the prompt to use device login to connect to Azure. Ignore any warnings about failing to load or parse files.

In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/Module_2_Get Started with AzureML/config.json
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FDATY94Y6 to authenticate.
Interactive authentication successfully completed.


Let's create an experiment. The experiment is bound to a workspace, and contains the methods to launch runs.

In [3]:
experiment = Experiment(workspace = ws, name = "my-first-experiment")

Next, let's start a run to estimate Pi value. While the experiment is running, we log metrics about the accuracy of the estimate into run history.

Let's also save the value of pi as a file, and upload that file into run history. In the next notebook we'll use the value in the file to create a web service that computes an area of a circle using our estimate.

In [4]:
run = experiment.start_logging()

pi_counter = 0
n_iter = 100000

# Log total number of iterations
run.log("Number of iterations",n_iter)

for i in range(1,n_iter):
    # Monte Carlo step to update estimate
    x = random.random()
    y = random.random()
    if x*x + y*y < 1.0:
        pi_counter += 1
    pi_estimate = 4.0*pi_counter / i
    
    # Log convergence every 10000 iterations
    if i%10000==0:
        error = math.pi-pi_estimate
        run.log("Pi estimate",pi_estimate)
        run.log("Error",error)

# Log final results
run.log("Final estimate",pi_estimate)
run.log("Final error",math.pi-pi_estimate)

# Write file containing pi value into run history
with open("pi_estimate.txt","wb") as f:
    pickle.dump(str(pi_estimate),f)
run.upload_file(name = 'outputs/pi_estimate.txt', path_or_stream = './pi_estimate.txt')

# Complete tracking and get link to details
run.complete()
print("Run completed")

Run completed


Once the run has completed, you can view a detailed report of the run from Azure Portal by simply calling "run" and following the link. You can view the convergence of estimate, as well as the uploaded file.

In [5]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
my-first-experiment,3f7bfc33-5163-45ca-bdba-d4b082cb3fa2,,Completed,Link to Azure Portal,Link to Documentation


You can also view all runs within an experiment. If you run the simulation above multiple times, these runs will appear under the experiment view and you can compare them.

In [6]:
experiment

Name,Workspace,Report Page,Docs Page
my-first-experiment,aiconfnycws,Link to Azure Portal,Link to Documentation


Next, learn how to deploy a web service that computes the area of circle using your estimate using following Notebook:

[Deploy web service](02.deploy-web-service.ipynb)

For an example using scikit-learn using Azure compute to train image classification model, see:

[tutorials/img-classification-part1-training](./tutorials/img-classification-part1-training.ipynb)